In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

In [2]:
pd.read_csv("magic04.data")

,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.011,-8.2027,40.092,81.8828,g
0,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
1,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
2,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
3,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g
4,51.6240,21.1502,2.9085,0.2420,0.1340,50.8761,43.1887,9.8145,3.6130,238.0980,g
...,...,...,...,...,...,...,...,...,...,...,...
19014,21.3846,10.9170,2.6161,0.5857,0.3934,15.2618,11.5245,2.8766,2.4229,106.8258,h
19015,28.9452,6.7020,2.2672,0.5351,0.2784,37.0816,13.1853,-2.9632,86.7975,247.4560,h
19016,75.4455,47.5305,3.4483,0.1417,0.0549,-9.3561,41.0562,-9.4662,30.2987,256.5166,h
19017,120.5135,76.9018,3.9939,0.0944,0.0683,5.8043,-93.5224,-63.8389,84.6874,408.3166,h


In [3]:
cols = ["fLength","fWidth","fSize","fConc","fConc1","fAsym","fM3Long","fM3Trans","fAlpha","fDist","class"]
df = pd.read_csv("magic04.data",names=cols)
df.head()

,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g


 here from "flength" to "class" called feature, feature are just thing that we are going to pass our model in order to help us predict the label,which in this case class column.

 here sample 0 has 10 different feature,so i have 10 different values that i can pass into some model.

In [ ]:
df["class"].unique()      #g-gamma h-hydra

In [4]:
df["class"]=(df['class']=='g').astype(int)

In [ ]:
df.head()

In [ ]:
df

In [ ]:
for labels in cols[:-1]:
  plt.hist(df[df["class"]==1][labels],color='blue',label='gamma',alpha=0.7,density=True)
  plt.hist(df[df["class"]==0][labels],color='red',label='hydron',alpha=0.7,density=True)
  plt.title(labels)
  plt.ylabel("Probability")
  plt.xlabel(labels)
  plt.legend()
  plt.show()

# Train Validate Test

In [5]:
train, validate, test = np.split(df.sample(frac=1), [int(0.6*len(df)), int(0.8*len(df))])      # splitting the dataframe, (df ...) this will shafapal my data, 0.6 means 60% data to validation, 0.8  means 80% data to test

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


if u seen in df.head() flength has over 100 but in fconc we have less than 100 ; the scale of numbers is way off and somethimes it will effect our results. so we need to scale this relative to mean and deviation of that specific column.

In [ ]:
def scale_dataset(dataframe):
  X = dataframe[dataframe.cols[:-1]].values
  y = dataframe[dataframe.cols[-1]].values

  # import standardScalar

  scaler = StandardScaler()
  X = scaler.fit_transform(X)

  # now we'll create the whole data as one 2d array to do that we need hstack
  # we are stacking x and y
  # numpy is very particular about dimensions
  # here x is 2d but y 1d(vector), now to reshape y to 2d, we need call reshape
  # and also need to pass dimensions
  # here (-1,1) means make this 2d array where -1 means len(y)

  data = np.hstack((X,np.reshape(y,(-1,1))))

  return data, X, y

In [ ]:
print(len(train[train["class"]==1]))  # gamma
print(len(train[train["class"]==0]))

here we have 7k gamma and 4k hydra, this will become a issue, so we need to oversample our training data sets means to increase the number of values, so they match with each other.

do reslove this we can import RandomOverSampler

In [6]:
def scale_dataset(dataframe, oversampler=False):
  X = dataframe[dataframe.columns[:-1]].values
  y = dataframe[dataframe.columns[-1]].values

  scaler = StandardScaler()
  X = scaler.fit_transform(X)

  if oversampler:
    ros = RandomOverSampler()
    X,y = ros.fit_resample(X,y)   # says take less class and keep sampling from their to increase the dataset of that smaller class, so that they can match

  data = np.hstack((X,np.reshape(y,(-1,1))))

  return data, X, y

In [ ]:
train, X_train, y_train = scale_dataset(train, oversampler= True)

In [ ]:
len(X_train)

In [ ]:
len(y_train)

In [ ]:
sum(y_train == 1)

In [ ]:
sum(y_train == 0)

why switching to false for validate and test bcz, i have a data that i haven't seen yet, how does my sample perform on those and i don't want to oversample those.I want ot know if i have a random set of data that's unlabeled, can i trust my model. that's why not oversampling.

In [7]:
train, X_train, y_train = scale_dataset(train, oversampler=True)
validate, X_validate, y_vaildate = scale_dataset(validate, oversampler=False)
test, X_test, y_test = scale_dataset(test, oversampler=False)

now we have our data properly formeted now will see different models.

# KNN or K Nearest Neighbors

Now we use packages from sklearn, why we use packages, so we don't have to manually code all these things by ourself.

In [11]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=1) # asks how many neighbors we want here i gave only one.
knn_model.fit(X_train,y_train)

In [ ]:
y_pred = knn_model.predict(X_test)

In [ ]:
y_pred

In [ ]:
y_test

In [ ]:
print(classification_report(y_pred,y_test))

accuracy is 82%, what it is saying is we just look at what each of these new points what it's closed to then we got 82% accuracy, which means how many do we get right versus how many total are there.

Now precision is we what saying class 0 and class 1, go to these site
https://en.wikipedia.org/wiki/Precision_and_recall

in the diagram, on left everything we know is actually trully +ve that we labeled +ve in our dataset and in right side is trully -ve.

Now in circle we have things +ve were labeled +ve by our model. on left side they should have been +ve but they were labelled as -ve by our model.

so precision is saying out of all the one labeled as positive, how many of them are true positive and recall saying out of all the ones labelled as +ve, how many do we actually get right.

f1-score is a combination of precision and recall score, we will be looking into this bcz we have unbalanced test data set.

#Naive Bayes

In [8]:
from sklearn.naive_bayes import GaussianNB

In [9]:
nb_model = GaussianNB()
nb_model.fit(X_train,y_train)

GaussianNB()

In [10]:
y_pred = nb_model.predict(X_test)

In [12]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.38      0.68      0.49       774
           1       0.90      0.72      0.80      3030

    accuracy                           0.71      3804
   macro avg       0.64      0.70      0.65      3804
weighted avg       0.79      0.71      0.74      3804



In [13]:
from sklearn.linear_model import LogisticRegression

In [15]:
lr=LogisticRegression()
lr.fit(X_train,y_train)

LogisticRegression()

In [16]:
y_pred=lr.predict(X_test)

In [17]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.70      0.68      0.69      1406
           1       0.82      0.83      0.82      2398

    accuracy                           0.78      3804
   macro avg       0.76      0.76      0.76      3804
weighted avg       0.78      0.78      0.78      3804

